In [1]:
import numpy as np
import pandas as pd

result = pd.read_csv('sample/preprocessed_dataset.csv')
print(result)
data = result.drop('파일명',axis=1)
data = data.drop('선종', axis=1)
data = data.drop('상태', axis=1)
data = data.drop('위치' , axis=1)
data

       파일명   선종    상태                               위치  state  vessel_type  \
0    1.jpg  범장망    표류    LAT N33.22.872 LON E124.5.074      3            4   
1    2.jpg  범장망    이동   LAT N32.48.241 LON E124.38.553      2            4   
2    3.jpg  범장망    조업   LAT N32.55.893 LON E124.33.731      1            4   
3    4.jpg   타망    이동   LAT N35.40.891 LON E124.25.481      2            1   
4    5.jpg   타망    이동   LAT N32.23.668 LON E125.33.053      2            1   
5    6.jpg   타망    이동   LAT N32.59.352 LON E124.29.609      2            1   
6    7.jpg   유망    조업   LAT N34.54.232 LON E124.42.394      1            2   
7    8.jpg   유망    조업   LAT N34.41.829 LON E124.20.120      1            2   
8    9.jpg   유망    표류   LAT N36.56.279 LON E124.39.427      3            2   
9   10.jpg  안강망    조업   LAT N34.15.543 LON E125.41.175      1            5   
10  11.jpg   통발  \t이동   LAT N34.17.855 LON E124.15.509      2            7   
11  12.jpg   통발    이동   LAT N33.41.223 LON E124.12.877      2   

,state,vessel_type,Latitude,Longitude,nation_type
0,3,4,3322872,1245074,0
1,2,4,3248241,12438553,0
2,1,4,3255893,12433731,0
3,2,1,3540891,12425481,0
4,2,1,3223668,12533053,0
5,2,1,3259352,12429609,0
6,1,2,3454232,12442394,0
7,1,2,3441829,12420120,0
8,3,2,3656279,12439427,0
9,1,5,3415543,12541175,1


In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical


# 이미지 디렉토리 경로
image_dir = '샘플데이터셋'  # 경로 수정 필요

# 이미지 읽기 및 전처리
images = []
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(image_dir, filename)
        image = Image.open(img_path).resize((1024, 1024))  # 이미지 크기 조정
        image_array = img_to_array(image) / 255.0  # 정규화
        images.append(image_array)

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 레이블을 정의 (예: 0 또는 1)
# 레이블을 적절히 설정하세요
labels = data['vessel_type']
labels = to_categorical(labels, num_classes=10)

# 모델 학습



2024-10-17 05:23:52.280452: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 05:23:54.181979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 05:23:54.637290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 05:23:54.777884: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 05:23:55.677923: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def MobileNetV2(input_shape=(1024, 1024, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2')
    return model

model = MobileNetV2()
model.summary()



/tmp/ipykernel_3391/2995523478.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
2024-10-17 05:24:03.360703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38716 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 3g.40gb, pci bus id: 0000:ca:00.0, compute capability: 8.0


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1024, 1024, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 32, 32, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 2,389,130 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
history = model.fit(images, labels, epochs=10, verbose=1)
predictions = model.predict(images)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)

Epoch 1/10


I0000 00:00:1729142668.491918    3512 service.cc:146] XLA service 0x7f4ee00044c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729142668.491950    3512 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe MIG 3g.40gb, Compute Capability 8.0
2024-10-17 05:24:29.029634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-17 05:24:32.149945: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
E0000 00:00:1729142681.301794    3512 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1729142681.471200    3512 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight

1/1 ━━━━━━━━━━━━━━━━━━━━ 82s 82s/step - accuracy: 0.1667 - loss: 2.6202
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.8333 - loss: 0.8902
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 1.0000 - loss: 0.3046
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 1.0000 - loss: 0.1947
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.9167 - loss: 0.2702
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 1.0000 - loss: 0.0996
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 1.0000 - loss: 0.0391
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 1.0000 - loss: 0.0217
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 1.0000 - loss: 0.0431
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 1.0000 - loss: 0.0214
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[4 4 4 4 1 4 4 4 4 1 1 4]


In [6]:
import folium

# 좌표 데이터 (위도, 경도)
locations = [
    {"name": "차", "lat": 32 + 11/60, "lon": 125 + 25/60},
    {"name": "카", "lat": 33 + 20/60, "lon": 124 + 8/60},
    {"name": "타", "lat": 34 + 0/60, "lon": 124 + 0/60 + 30/3600},
    {"name": "파", "lat": 35 + 0/60, "lon": 124 + 7/60 + 30/3600},
    {"name": "하", "lat": 35 + 30/60, "lon": 124 + 30/60},
    {"name": "거", "lat": 36 + 45/60, "lon": 124 + 30/60},
    {"name": "너", "lat": 37 + 0/60, "lon": 124 + 20/60},
    {"name": "바", "lat": 32 + 11/60, "lon": 126 + 45/60},
]

# 초기 지도의 중심 좌표 (평균 값 사용)
center_lat = sum([loc["lat"] for loc in locations]) / len(locations)
center_lon = sum([loc["lon"] for loc in locations]) / len(locations)

# 지도 생성
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# 각 위치에 마커 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=loc["name"]
    ).add_to(mymap)

# 지도 저장
mymap.save("map.html")

# 지도를 출력하고 싶다면, 주피터 노트북 환경에서는 다음 코드로 출력 가능
mymap


ModuleNotFoundError: No module named 'folium'

In [8]:
pip install pytesseract opencv-python pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
